In [76]:
import requests
import re
from datetime import datetime
from collections import Counter

# Replace with your actual Etherscan API key
ETHERSCAN_API_KEY = "EVWY88Y9UDYU4JYTBFHRN7WNPVA253YRTA"
CHAINABUSE_API_KEY= " Y2FfTVV4SVZUTmhPRlpwV0UxVFJFOXphamRsV0VwelYzWXdMa3QwTlVwMFRIbzNSa0pXV2xvd00xcFJaMG9yYTNjOVBROkk2RE9OVDA5S05PVyM="


def is_ethereum_address(address):
    """Checks if the given address is a valid Ethereum address."""
    return bool(re.match(r"^0x[a-fA-F0-9]{40}$", address))

def is_bitcoin_address(address):
    """Checks if the given address is a valid Bitcoin address."""
    return bool(re.match(r"^[13][a-km-zA-HJ-NP-Z1-9]{25,34}$", address))

def check_bitcoin_scam_activity(address):
    """Fetches Bitcoin transactions and detects scam-related activity."""
    url = f"https://blockchain.info/rawaddr/{address}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        total_inputs = 0
        total_outputs = 0
        large_transactions = 0
        high_recipient_txs = 0

        HIGH_RECIPIENT_THRESHOLD = 10  # More than 10 recipients
        LARGE_TX_THRESHOLD = 1_000_000  # 1 BTC in satoshis

        for tx in data.get("txs", []):
            inputs = len(tx.get("inputs", []))
            outputs = len(tx.get("out", []))

            total_inputs += inputs
            total_outputs += outputs

            if outputs > HIGH_RECIPIENT_THRESHOLD:
                high_recipient_txs += 1

            for output in tx.get("out", []):
                if output.get("value", 0) > LARGE_TX_THRESHOLD:
                    large_transactions += 1

        if total_inputs == 0 and total_outputs == 0:
            print(f"🔍 BTC Address {address}: No transactions found.")
            return

        print(f"\n🔍 BTC Address: {address}")
        print(f"   📥 Total Inward Transfers: {total_inputs}")
        print(f"   📤 Total Outward Transfers: {total_outputs}")
        print(f"   💰 Large Transactions (>1 BTC): {large_transactions}")
        print(f"   🔀 Transactions with Many Recipients (>10): {high_recipient_txs}")

        if high_recipient_txs > 2 or large_transactions > 2:
            print("   ⚠️ Potential Scam Detected!")
        else:
            print("   ✅ Legitimate Activity Detected.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for BTC address {address}: {e}")

def check_ethereum_scam_activity(address):
    """Fetches Ethereum transactions and detects scam-related activity."""
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=asc&apikey={ETHERSCAN_API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if data.get("status") != "1":
            print(f"🔍 ETH Address {address}: No transactions found or invalid address.")
            return

        transactions = data.get("result", [])
        total_inward = 0
        total_outward = 0
        large_transactions = 0
        high_recipient_txs = 0

        HIGH_RECIPIENT_THRESHOLD = 10  # More than 10 recipients
        LARGE_TX_THRESHOLD = 1 * 10**18  # 1 ETH in Wei

        for tx in transactions:
            from_address = tx.get("from", "").lower()
            to_address = tx.get("to", "").lower()
            value = int(tx.get("value", 0))

            if to_address == address.lower():
                total_inward += 1
            if from_address == address.lower():
                total_outward += 1

                if value > LARGE_TX_THRESHOLD:
                    large_transactions += 1

        if total_inward == 0 and total_outward == 0:
            print(f"🔍 ETH Address {address}: No transactions found.")
            return

        print(f"\n🔍 ETH Address: {address}")
        print(f"   📥 Total Inward Transfers: {total_inward}")
        print(f"   📤 Total Outward Transfers: {total_outward}")
        print(f"   💰 Large Transactions (>1 ETH): {large_transactions}")
        print(f"   🔀 Transactions with Many Recipients (>10): {high_recipient_txs}")

        if large_transactions > 2 or high_recipient_txs > 2:
            print("   ⚠️ Potential Scam Detected!")
        else:
            print("   ✅ Legitimate Activity Detected.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for ETH address {address}: {e}")


def fetch_chainabuse_reports(address):#link to fetch the data
    url = f"https://api.chainabuse.com/v0/reports?blockchain={blockchain_type}&address={scam_address}&includePrivate=false&page=1&perPage=50"
    
    #it contains the API Key (authentication details)
    headers = {
    "accept": "application/json",
    "authorization": f"Basic {CHAINABUSE_API_KEY}" 
}
    try: 
        response = requests.get(url, headers=headers) #to request API 
        if response.status_code == 429:#to checks whether the API request was successful or not, as there's limits, 10/month
            print ("Limit exceeded")
            return None   # Return None when the API rate limit is exceeded
    
        response.raise_for_status() #if the request is successful, nothing happen, if the request fails, it raises an exception
        data = response.json()  # Convert response to JSON format
        if isinstance(data, list):#if the data is in list
            reports = data  # If the entire response is a list, it is assigned into reports
            reports = sorted(reports, key= lambda x: datetime.strptime(x['createdAt'], '%Y-%m-%dT%H:%M:%S.%fZ')) #sort from the earliest to the latest date of report
            
        if not isinstance(data, list) or len(data) == 0: #if it's not a list or there's nothing inside data then it return there's no scam
            print(f"❌ No scam reports found for {address}.")
            return None
        
        dates = []
        scam_categories = []
        
        
        for i, report in enumerate(data, start=1): 
            created_at = report.get('createdAt') #to fetch the createdAt from report, if it's NA's then it will returns None
            scam_category = report.get('scamCategory', 'Not Available') #to get the scamCategory from the report, if the key 'scamCategory' is not exist, it return Not Available
            
            if created_at: #This conditional checks if the created_at value is not None or an empty string.
                dates.append(datetime.strptime(created_at, '%Y-%m-%dT%H:%M:%S.%fZ')) #it convert the created_at into date-time format
            
            scam_categories.append(scam_category)#add the scam_category into scam_categories
        
        # Get earliest & latest date
        start_date = min(dates).strftime('%Y-%m-%d') if dates else "Unknown" #This finds the earliest (smallest) datetime in the "dates",  convert it into year month date format, if the dates is empty then return Unknown
        latest_date = max(dates).strftime('%Y-%m-%d') if dates else "Unknown"#this to find the latest datetime of scam, if it's not available return unknown
        
        # Get the most frequent scam category
        most_frequent_category = Counter(scam_categories).most_common(1)[0][0] if scam_categories else "Unknown" #to get the most scam category reported, which is "1" meaning the most common elements, [0][0] to take the first tuple, if not available return Unknown
        
        num_reports = len(data) # Count the total number of reports
        
        

        print(f" 🚨  Scam reports summary for {address}:")
        print(f"        Total Reports: {num_reports}")
        print(f"        First Report: {start_date}")
        print(f"        Latest Report: {latest_date}")
        print(f"        Most Frequent Scam Type: {most_frequent_category}")

        

        return start_date, latest_date, most_frequent_category
            
            
    except requests.exceptions.RequestException as e: #if an error occurs it prints an error message with details
        print(f"Error fetching scam reports for {address}: {e}")
        return False


if __name__ == "__main__":
    user_input = input("Enter Bitcoin or Ethereum address(es) (comma-separated for multiple): ")
    addresses = [addr.strip() for addr in user_input.split(",")]

    for address in addresses:
        if is_bitcoin_address(address):
            scam_report = fetch_chainabuse_reports(address)  # First check for scam reports 
            check_bitcoin_scam_activity(address)
        elif is_ethereum_address(address):
            scam_report = fetch_chainabuse_reports(address)  # First check for scam reports
            
            check_ethereum_scam_activity(address)
        else:
            print(f"❌ Invalid address format: {address}")


 🚨  Scam reports summary for 135qVXXBZb3v2tQcLJRA8UAndiUYNybh3J:
        Total Reports: 50
        First Report: 2018-10-08
        Latest Report: 2019-01-14
        Most Frequent Scam Type: OTHER_BLACKMAIL

🔍 BTC Address: 135qVXXBZb3v2tQcLJRA8UAndiUYNybh3J
   📥 Total Inward Transfers: 118
   📤 Total Outward Transfers: 138
   💰 Large Transactions (>1 BTC): 118
   🔀 Transactions with Many Recipients (>10): 2
   ⚠️ Potential Scam Detected!
